In [1]:
# ============================================================
# Global Wheat Detection: YOLOv8 vs Faster R-CNN  
# ============================================================

# --- Clean & pin compatible packages ---
!pip -q uninstall -y ray ray-core ray[default] || true      # remove Ray to avoid Ultralytics callback crash
!pip -q uninstall -y albumentations albucore -y             # remove conflicting installs of albumentations/albucore
!pip -q install --upgrade "ultralytics==8.3.29" "pycocotools" "torchmetrics" \
                             "opencv-python-headless==4.10.0.84" \
                             "albumentations==1.3.1"         # albumentations pre-albucore split

# --- Imports & set env flags ---
import os, gc, re, ast, time, random, shutil             # stdlib helpers
from pathlib import Path                                  # path ops
import numpy as np                                        # arrays
import pandas as pd                                       # tables
from PIL import Image, ImageDraw                          # image IO/annotation
from tqdm.auto import tqdm                                 # progress bars

os.environ["ULTRALYTICS_RAY"] = "0"                       # hard-disable Ray callbacks in Ultralytics

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision.transforms import functional as TF

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from ultralytics import YOLO

# --------------------------
# Reproducibility & device
# --------------------------
SEED = 42                                                 # random seed for reproducibility
random.seed(SEED); np.random.seed(SEED)
torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# --------------------------
# Config section given below
# --------------------------
EPOCHS_YOLO   = 20      
EPOCHS_FRCNN  = 6      # making it 6 to complete quick
BATCH_FRCNN   = 2      # small batch -> less VRAM
IMGSZ         = 640    # smaller images than 1024 to save RAM/VRAM
NUM_WORKERS   = 1      # 1 worker keeps RAM usage modest
VAL_FRACTION  = 0.2    # 80/20 split
N_QUAL        = 8      # number of qualitative samples to save
SPEED_SAMPLES = 60     # images for speed tests
CLASS_NAME    = "wheat_head"  # single-class detection

# --- Working directories ---
WORKDIR        = Path("/kaggle/working")
YOLO_DIR       = WORKDIR / "wheat_yolo"
YOLO_IMG_DIR_TR= YOLO_DIR/"images/train"
YOLO_IMG_DIR_VA= YOLO_DIR/"images/val"
YOLO_LAB_DIR_TR= YOLO_DIR/"labels/train"
YOLO_LAB_DIR_VA= YOLO_DIR/"labels/val"
OUT_DIR        = WORKDIR / "outputs"
for p in [YOLO_IMG_DIR_TR, YOLO_IMG_DIR_VA, YOLO_LAB_DIR_TR, YOLO_LAB_DIR_VA, OUT_DIR]:
    p.mkdir(parents=True, exist_ok=True)                  # create folders

# --------------------------
# Locate dataset in Kaggle
# --------------------------
def find_wheat_input():
    """Return path to Global Wheat Detection dataset inside /kaggle/input."""
    cand = Path("/kaggle/input/global-wheat-detection")
    if cand.exists(): return cand
    for d in Path("/kaggle/input").glob("*"):
        if (d/"train.csv").exists() and (d/"train").exists():
            return d
    raise FileNotFoundError("Global Wheat Detection dataset not found under /kaggle/input")

DATA_ROOT    = find_wheat_input()
print("Dataset root:", DATA_ROOT)

TRAIN_IMG_DIR= DATA_ROOT / "train"   # training images
TRAIN_CSV    = DATA_ROOT / "train.csv"# annotations (bbox per row)
TEST_IMG_DIR = DATA_ROOT / "test"    # optional visuals

# --- Load annotations ---
df = pd.read_csv(TRAIN_CSV)                      # read bbox file
print("Train CSV shape:", df.shape)
display(df.head())

# --------------------------
# Parse bbox strings -> [x,y,w,h]
# --------------------------
def parse_bbox_str(s):
    """Parse '[x, y, w, h]' strings robustly into list of floats."""
    s = str(s).strip()
    if s.startswith("[") and s.endswith("]"):
        return list(map(float, ast.literal_eval(s)))
    nums = re.findall(r"[-+]?\d*\.\d+|\d+", s)   # fallback: regex numbers
    return list(map(float, nums[:4]))

df["bbox_parsed"] = df["bbox"].apply(parse_bbox_str)   # convert bbox column
img_meta = df.groupby("image_id")[["width","height"]].first().to_dict("index")  # size per image

# --- Train/Val split by image_id to avoid leak ---
image_ids = list(img_meta.keys()); random.shuffle(image_ids)
n_val     = int(len(image_ids)*VAL_FRACTION)
val_ids   = set(image_ids[:n_val])
train_ids = set(image_ids[n_val:])
print("Train images:", len(train_ids), "Val images:", len(val_ids))

# ======================================================
# A) Prepare YOLO format (txt per image: cls xc yc w h)
# ======================================================
def coco_to_yolo(xywh, img_w, img_h):
    """COCO [x,y,w,h] -> YOLO normalized [xc,yc,w,h]."""
    x, y, w, h = xywh
    xc = x + w/2.0; yc = y + h/2.0
    return [xc/img_w, yc/img_h, w/img_w, h/img_h]

def write_yolo_label(lbl_path, rows, img_w, img_h):
    """Write a YOLO .txt with one line per bbox: '0 xc yc w h'."""
    with open(lbl_path, "w") as f:
        for xywh in rows:
            xc,yc,ww,hh = coco_to_yolo(xywh, img_w, img_h)
            f.write(f"0 {xc:.6f} {yc:.6f} {ww:.6f} {hh:.6f}\n")

def prepare_yolo_split(split_ids, img_target_dir, lab_target_dir):
    """Symlink images and emit YOLO label files for a set of image_ids."""
    count = 0
    for img_id in tqdm(split_ids, desc=f"YOLO prep -> {img_target_dir.parent.name}"):
        src = TRAIN_IMG_DIR/f"{img_id}.jpg"
        if not src.exists(): 
            continue
        dst_img = img_target_dir/f"{img_id}.jpg"
        if not dst_img.exists():
            os.symlink(src, dst_img)                       # link image (fast, low disk)
        rows = df[df.image_id==img_id]["bbox_parsed"].tolist()
        imw, imh = img_meta[img_id]["width"], img_meta[img_id]["height"]
        write_yolo_label(lab_target_dir/f"{img_id}.txt", rows, imw, imh)
        count += 1
    return count

# --- Build YOLO train/val folders & labels ---
n_tr = prepare_yolo_split(train_ids, YOLO_IMG_DIR_TR, YOLO_LAB_DIR_TR)
n_va = prepare_yolo_split(val_ids,   YOLO_IMG_DIR_VA, YOLO_LAB_DIR_VA)
print(f"YOLO files -> train:{n_tr}, val:{n_va}")

# --- YOLO data.yaml describing paths/class names ---
(DATA_YAML := YOLO_DIR/"wheat.yaml").write_text(f"""\
path: {YOLO_DIR.as_posix()}
train: images/train
val: images/val
nc: 1
names: ['{CLASS_NAME}']
""")
print("Wrote", DATA_YAML)

# ======================================================
# B) YOLOv8 training, validation, speed, qualitative
# ======================================================
yolo_model = YOLO("yolov8n.pt")  # small model for speed; try 'yolov8s.pt' later

# --- Train YOLO (cache to disk to reduce RAM) ---
yolo_results = yolo_model.train(
    data=str(DATA_YAML),
    epochs=EPOCHS_YOLO,
    imgsz=IMGSZ,
    cache="disk",
    batch=8,
    workers=2,
    device=0 if device.type=="cuda" else "cpu",
    project=str(WORKDIR/"ultra_runs"),
    name="yolo_wheat",
    patience=3,                           # early stop if no improvement
    verbose=True
)

# --- Best checkpoint path (for logs/info only) ---
best_dir = Path(yolo_results.save_dir)
best_pt  = best_dir/"weights/best.pt"
print("YOLO best:", best_pt if best_pt.exists() else "unknown")

# --- Validate YOLO & extract mAP metrics robustly across versions ---
yolo_map50 = np.nan; yolo_map5095 = np.nan
try:
    yres = yolo_model.val(workers=2, imgsz=IMGSZ, plots=False, verbose=False)
    if hasattr(yres, "metrics") and hasattr(yres.metrics, "box"):
        yolo_map50   = float(getattr(yres.metrics.box, "map50", np.nan))
        yolo_map5095 = float(getattr(yres.metrics.box, "map",   np.nan))
    elif hasattr(yres, "results_dict"):
        rd = yres.results_dict
        yolo_map50   = float(rd.get("metrics/mAP50(B)",     np.nan))
        yolo_map5095 = float(rd.get("metrics/mAP50-95(B)",  np.nan))
except Exception as e:
    print("YOLO val metric parse warning:", e)

print("YOLO mAP50:", yolo_map50, "mAP50-95:", yolo_map5095)

# --- Measure YOLO inference speed on a subset ---
def measure_yolo_speed(model, img_dir, num=60):
    paths = list(Path(img_dir).glob("*.jpg"))
    if not paths: 
        return np.nan, np.nan
    paths = random.sample(paths, min(num, len(paths)))
    _ = model.predict(source=str(paths[0]), imgsz=IMGSZ, verbose=False)  # warmup
    t0 = time.time()
    _  = model.predict(source=[str(p) for p in paths], imgsz=IMGSZ, verbose=False)
    dt = time.time()-t0
    tpi = dt/len(paths)
    return 1.0/tpi, tpi

yolo_fps, yolo_tpi = measure_yolo_speed(yolo_model, YOLO_IMG_DIR_VA, num=SPEED_SAMPLES)
print(f"YOLO speed -> {yolo_fps:.2f} FPS ({yolo_tpi*1000:.1f} ms/img)")

# --- Save YOLO qualitative predictions (annotated images) ---
(yolo_pred_dir := OUT_DIR/"qual_yolo").mkdir(exist_ok=True, parents=True)
_ = yolo_model.predict(source=str(YOLO_IMG_DIR_VA), save=True, project=str(yolo_pred_dir),
                       name="preds", imgsz=IMGSZ, max_det=300, conf=0.25, verbose=False)
print("YOLO qualitative saved to:", yolo_pred_dir)

# --- Free GPU/CPU RAM before Faster R-CNN ---
del yolo_results; gc.collect()
try:
    if torch.cuda.is_available(): torch.cuda.empty_cache()
except: pass

# ======================================================
# C) Faster R-CNN dataset, training, eval, speed, qualitative
# ======================================================
class WheatDataset(Dataset):
    """Minimal detection dataset for torchvision's Faster R-CNN."""
    def __init__(self, ids, img_dir, df, transforms=None):
        self.ids = list(ids)
        self.img_dir = Path(img_dir)
        self.df = df
        self.transforms = transforms
        self.group = df.groupby("image_id")["bbox_parsed"].apply(list).to_dict()

    def __len__(self): 
        return len(self.ids)

    def __getitem__(self, idx):
        img_id  = self.ids[idx]
        img_path= self.img_dir/f"{img_id}.jpg"
        image   = np.array(Image.open(img_path).convert("RGB"))
        boxes   = np.array(self.group[img_id], dtype=np.float32)   # [x,y,w,h]

        # convert to [x1,y1,x2,y2] as expected by torchvision
        boxes_xyxy      = boxes.copy()
        boxes_xyxy[:,2] = boxes[:,0] + boxes[:,2]
        boxes_xyxy[:,3] = boxes[:,1] + boxes[:,3]
        boxes_xyxy[:,0] = boxes[:,0]
        boxes_xyxy[:,1] = boxes[:,1]
        labels          = np.ones((len(boxes_xyxy),), dtype=np.int64)  # single class=1

        target = {
            "boxes":    torch.as_tensor(boxes_xyxy, dtype=torch.float32),
            "labels":   torch.as_tensor(labels,      dtype=torch.int64),
            "image_id": torch.tensor([idx])
        }

        if self.transforms:
            transformed = self.transforms(image=image, bboxes=boxes_xyxy, class_labels=labels.tolist())
            image = transformed["image"]
            bxs   = transformed["bboxes"]; cl = transformed["class_labels"]
            if len(bxs) == 0:
                target["boxes"]  = torch.zeros((0,4), dtype=torch.float32)
                target["labels"] = torch.zeros((0,),   dtype=torch.int64)
            else:
                target["boxes"]  = torch.tensor(bxs, dtype=torch.float32)
                target["labels"] = torch.tensor(cl,  dtype=torch.int64)
        else:
            image = TF.to_tensor(Image.fromarray(image))
        return image, target

# --- Albumentations transforms — ensure float tensors for torchvision ---
train_tfms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.15),
    A.LongestMaxSize(max_size=IMGSZ, p=1.0),
    A.PadIfNeeded(min_height=IMGSZ, min_width=IMGSZ, border_mode=0, value=(114,114,114)),
    A.ToFloat(max_value=255.0),     # convert to float [0,1]
    ToTensorV2()
], bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]))

val_tfms = A.Compose([
    A.LongestMaxSize(max_size=IMGSZ, p=1.0),
    A.PadIfNeeded(min_height=IMGSZ, min_width=IMGSZ, border_mode=0, value=(114,114,114)),
    A.ToFloat(max_value=255.0),     # convert to float [0,1]
    ToTensorV2()
], bbox_params=A.BboxParams(format="pascal_voc", label_fields=["class_labels"]))

# --- Build datasets/dataloaders (memory-conscious loader settings) ---
ds_train = WheatDataset(train_ids, TRAIN_IMG_DIR, df, transforms=train_tfms)
ds_val   = WheatDataset(val_ids,   TRAIN_IMG_DIR, df, transforms=val_tfms)

def collate_fn(batch): 
    return tuple(zip(*batch))        # torchvision detection expects lists, not tensors

dl_train = DataLoader(ds_train, batch_size=BATCH_FRCNN, shuffle=True,
                      num_workers=NUM_WORKERS, collate_fn=collate_fn,
                      pin_memory=False, persistent_workers=False)
dl_val   = DataLoader(ds_val,   batch_size=BATCH_FRCNN, shuffle=False,
                      num_workers=NUM_WORKERS, collate_fn=collate_fn,
                      pin_memory=False, persistent_workers=False)

# --- Model: Faster R-CNN ResNet50 FPN, replace head for 2 classes (bg + wheat) ---
frcnn = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
in_feats = frcnn.roi_heads.box_predictor.cls_score.in_features
frcnn.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_feats, 2)
frcnn = frcnn.to(device)

# --- Optimizer & LR schedule ---
params       = [p for p in frcnn.parameters() if p.requires_grad]
optimizer    = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

# --- Train loop (ensure float tensors) ---
def train_frcnn(model, dl, epochs):
    model.train()
    for epoch in range(epochs):
        losses_epoch = []
        pbar = tqdm(dl, desc=f"FRCNN Epoch {epoch+1}/{epochs}")
        for images, targets in pbar:
            images  = [img.float().to(device) for img in images]  # make sure images are float32
            targets = [{k: v.to(device) for k,v in t.items()} for t in targets]

            loss_dict = model(images, targets)            # returns dict of losses
            loss = sum(loss_dict.values())                # scalar total loss

            optimizer.zero_grad(); loss.backward(); optimizer.step()
            losses_epoch.append(loss.item())
            pbar.set_postfix(loss=np.mean(losses_epoch))
        lr_scheduler.step()
        print(f"Epoch {epoch+1} mean loss: {np.mean(losses_epoch):.4f}")

train_frcnn(frcnn, dl_train, EPOCHS_FRCNN)

# --- Evaluate mAP (0.5 and 0.5:0.95) with torchmetrics ---
@torch.no_grad()
def eval_map(model, dl):
    model.eval()
    metric = MeanAveragePrecision(iou_type="bbox")
    for images, targets in tqdm(dl, desc="Eval FRCNN"):
        images_gpu = [img.float().to(device) for img in images]  # float
        preds = model(images_gpu)
        preds_cpu, targets_cpu = [], []
        for p, t in zip(preds, targets):
            preds_cpu.append({
                "boxes":  p["boxes"].detach().cpu(),
                "scores": p["scores"].detach().cpu(),
                "labels": p["labels"].detach().cpu()
            })
            targets_cpu.append({
                "boxes":  t["boxes"].cpu(),
                "labels": t["labels"].cpu()
            })
        metric.update(preds_cpu, targets_cpu)
    res = metric.compute()
    return float(res["map_50"]), float(res["map"])

frcnn_map50, frcnn_map5095 = eval_map(frcnn, dl_val)
print("FRCNN mAP50:", frcnn_map50, "mAP50-95:", frcnn_map5095)

# --- Measure Faster R-CNN inference speed ---
@torch.no_grad()
def frcnn_speed(model, ds, num=60):
    model.eval()
    idxs = random.sample(range(len(ds)), min(num, len(ds)))
    img,_ = ds[idxs[0]]; _ = model([img.float().to(device)])  # warmup
    t0 = time.time()
    for i in idxs:
        img,_ = ds[i]; _ = model([img.float().to(device)])
    dt  = time.time() - t0
    tpi = dt/len(idxs)
    return 1.0/tpi, tpi

frcnn_fps, frcnn_tpi = frcnn_speed(frcnn, ds_val, num=SPEED_SAMPLES)
print(f"FRCNN speed -> {frcnn_fps:.2f} FPS ({frcnn_tpi*1000:.1f} ms/img)")

# --- Save qualitative predictions for Faster R-CNN ---
def draw_boxes(img_path, boxes, scores=None, thr=0.25):
    """Draw rectangles for predicted boxes (optionally filtered by score)."""
    im = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(im)
    for i, b in enumerate(boxes):
        if scores is not None and scores[i] < thr: 
            continue
        x1,y1,x2,y2 = map(float, b)
        draw.rectangle([x1,y1,x2,y2], outline=(0,255,0), width=2)
    return im

(qual_dir_frcnn := OUT_DIR/"qual_frcnn").mkdir(parents=True, exist_ok=True)

@torch.no_grad()
def save_qualitative(model, ids, n=8, outdir=qual_dir_frcnn):
    """Save n validation images with predicted boxes."""
    sel = random.sample(list(ids), min(n, len(ids)))
    model.eval()
    for img_id in sel:
        p   = TRAIN_IMG_DIR/f"{img_id}.jpg"
        img = TF.to_tensor(Image.open(p).convert("RGB")).float().to(device)
        pred= model([img])[0]
        im  = draw_boxes(p, pred["boxes"].detach().cpu().numpy(),
                         pred["scores"].detach().cpu().numpy(), thr=0.25)
        im.save(outdir/f"{img_id}_pred.jpg")

save_qualitative(frcnn, val_ids, n=N_QUAL)
print("FRCNN qualitative saved to:", qual_dir_frcnn)

# ======================================================
# D) Final comparison table 
# ======================================================
comp = pd.DataFrame([
    {"Model":"YOLOv8n",
     "mAP@0.5": round(yolo_map50, 4),
     "mAP@0.5:0.95": round(yolo_map5095, 4),
     "Time/Image (s)": round(yolo_tpi, 4),
     "FPS": round(yolo_fps, 2)},
    {"Model":"Faster R-CNN (R50-FPN)",
     "mAP@0.5": round(frcnn_map50, 4),
     "mAP@0.5:0.95": round(frcnn_map5095, 4),
     "Time/Image (s)": round(frcnn_tpi, 4),
     "FPS": round(frcnn_fps, 2)}
])
print("\n=== Results Comparison ===")
print(comp.to_string(index=False))
OUT_DIR.mkdir(exist_ok=True, parents=True)
comp.to_csv(OUT_DIR/"comparison.csv", index=False)
print("\nSaved comparison CSV to:", OUT_DIR/"comparison.csv")
print("\nQualitative results:\n - YOLOv8:", OUT_DIR/'qual_yolo', "\n - FRCNN  :", OUT_DIR/'qual_frcnn')



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.8/883.8 kB 16.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 34.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


Train images: 2699 Val images: 674


YOLO prep -> images:   0%|          | 0/2699 [00:00<?, ?it/s]

YOLO prep -> images:   0%|          | 0/674 [00:00<?, ?it/s]

...

FRCNN Epoch 6/6:   0%|          | 0/1350 [00:00<?, ?it/s]

Epoch 6 mean loss: 0.6738


Eval FRCNN:   0%|          | 0/337 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d9c36e116c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d9c36e116c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 16

FRCNN mAP50: 0.922092854976654 mAP50-95: 0.5110437870025635
FRCNN speed -> 8.82 FPS (113.4 ms/img)
FRCNN qualitative saved to: /kaggle/working/outputs/qual_frcnn

=== Results Comparison ===
                 Model  mAP@0.5  mAP@0.5:0.95  Time/Image (s)   FPS
               YOLOv8n   0.9391        0.5525          0.0336 29.78
Faster R-CNN (R50-FPN)   0.9221        0.5110          0.1134  8.82

Saved comparison CSV to: /kaggle/working/outputs/comparison.csv

Qualitative results:
 - YOLOv8: /kaggle/working/outputs/qual_yolo 
 - FRCNN  : /kaggle/working/outputs/qual_frcnn
